In [106]:
import numpy as np
from scipy.sparse import csr_matrix
import mdptoolbox

Constants:

In [107]:
# Switch to 2x2 flag
# The code is written generically and originally designed for 3x3
# with a lot of work going into solving the memory issues by sparse matrices and so on
# but the mdp algorithms just dont terminate on my machine with 3x3
# So in the end I decided to introduce this flag to allow evaluation
TWO_BY_TWO = True

S = 1_572_864
A = 19
IGNORE_ACTION = 18
COLORS = 3
OPERATIONS = 2 * COLORS
STORE = 1
COLOR_EMPTY = 3

if TWO_BY_TWO:
    A = 9
    IGNORE_ACTION = 8
    S = int(S / (4 ** 5))

# Same props for store and restore, different props for colors
COLOR_PROPS = np.array([.25, .15, .1, .25, .15, .1])

For easier calculation the states and actions are binary mask encoded
State Encoding
The first 18 bits (0-17) are used for the storage state. 2 bits for each of the 9 squares.
The 18th bit is used for the type of operation: store/restore.
And the 19th and 20th bit to specify the color to be stored/restored.
Action Encoding:
The first bit specifies the type of operation: store/restore
The following four bits specify the square to perform the operation on
State IGNORE_ACTION is used for rejecting

For 2x2:
3 bits are enough for action encoding
the offset for the game state changes
basically everything changes

Helpers for state manipulation and read out:

In [108]:
def get_action(a):
    # ACHTUNG: IGNORE_ACTION needs to be handled separately
    action = a & 1  # first bit: store/restore
    a = a >> 1

    if not TWO_BY_TWO:
        # Convert 0...8 to 0..2 x 0..2
        x = a % 3
        y = int(a / 3)
    else:
        x = a & 1
        y = (a >> 1) & 1

    return action, x, y

def get_task(s):
    if not TWO_BY_TWO:
        task = s & (0b1 << 18)
        color = s & (0b11 << 19)
    else:
        task = s & (0b1 << 8)
        color = s & (0b11 << 9)
    return task, color

def get_color(s, x, y):
    if not TWO_BY_TWO:
        pos = 3*y + x
    else:
        pos = 2*y + x
    mask = 0b11 << (pos * 2)
    color = s & mask
    return color


def set_color(s, x, y, color):
    if not TWO_BY_TWO:
        pos = 3*y + x
    else:
        pos = 2*y + x

    # Reset color
    mask = 0b11 << (pos * 2)
    s = s & (~mask)

    # Set color
    mask = color << (pos * 2)
    s = s & mask

    return s

Initialization of the probability matrix P.

In [109]:
# number of actions sized ndarrays
if not TWO_BY_TWO:
    P = np.ndarray((A,), dtype=object)

else:
    P = np.zeros((A,S,S))

# for every action
for a in range(A):
    # create a state x state sparse transition probability matrix
    row_indices = []
    col_indices = []
    data = []
    # for every state
    for s in range(S):
        ### apply storage state change ###
        action, x, y = get_action(a)

        # Case Ignore
        if a == IGNORE_ACTION:
            state = s

        # Case Store
        elif action == STORE:
            if not get_color(s, x, y) == COLOR_EMPTY:
                state = s
            else:
                task, color = get_task(s)
                state = set_color(s, x, y, color)
        else:
            # Case restore
            state = set_color(s, x, y, COLOR_EMPTY)

        ### get all possible state mutations for different following tasks ###
        # Reset task in state
        if not TWO_BY_TWO:
            mask = ~(0b111 << 18)
        else:
            mask = ~(0b111 << 8)
        state = state & mask

        # get all permutations of task
        mutations = []
        for i in range(COLORS * 2):
            if not TWO_BY_TWO:
                mask = i << 18
            else:
                mask = i << 8
            mutations.append(state | mask)

        # add all state transition probabilities
        for i, mut in enumerate(mutations):
            if not TWO_BY_TWO:
                row_indices.append(s)
                col_indices.append(mut)
                data.append(COLOR_PROPS[i])
            else:
                P[a][s][mut] = COLOR_PROPS[i]
    if not TWO_BY_TWO:
        P[a] = csr_matrix((data, (row_indices, col_indices)), shape=(S, S))
    print("Progress: " + str(a+1) + "/" + str(A))


Progress: 1/9
Progress: 2/9
Progress: 3/9
Progress: 4/9
Progress: 5/9
Progress: 6/9
Progress: 7/9
Progress: 8/9
Progress: 9/9


Initialization of the risk matrix R.

In [110]:
R = np.ndarray((S, A))

# For every possible action
for a in range(A):
    # For every possible state
    for s in range(S):

        # extract action, x and y
        action, x, y = get_action(a)

        # extract task and color
        task, color = get_task(s)

        if a == IGNORE_ACTION:
            R[s][a] = -10
            continue

        if not action == task: # Wrong action: illegal
            R[s][a] = -100
            continue

        if action == STORE:
            if not get_color(s, x, y) == COLOR_EMPTY: # Tried storing to already filled square: illegal
                R[s][a] = -100
                continue
            # Else: legal storing move
            # Penalize for distance only
            R[s][a] = -(1+x+y)
            continue

        # Else: Action is restore
        if get_color(s, x, y) == COLOR_EMPTY: # Nothing there to restore: illegal
            R[s][a] = -100
            continue

        # Else: Legal Restoring
        if not get_color(s, x, y) == color: # restored the wrong thing: illegal
            R[s][a] = -100
            continue

        # Else: perfectly restored
        # Penalize for distance only
        R[s][a] = -(1+x+y)
    print("Progress: " + str(a+1) + "/" + str(A))

Progress: 1/9
Progress: 2/9
Progress: 3/9
Progress: 4/9
Progress: 5/9
Progress: 6/9
Progress: 7/9
Progress: 8/9
Progress: 9/9


Learn policy by Q-Learning, Value Iteration, Policy Iteration and Policy Iteration GS

In [111]:
mdp = mdptoolbox.mdp.QLearning(P, R, 0.99)
mdp.run()

mdp_policy_iter = mdptoolbox.mdp.PolicyIteration(P, R, .99)
mdp_policy_iter.run()

mdp_value_iter = mdptoolbox.mdp.ValueIteration(P, R, .99)
mdp_value_iter.run()

mpd_value_iter_gs = mdptoolbox.mdp.ValueIterationGS(P, R, .99)
mpd_value_iter_gs.run()

mdp_rel_value_iter = mdptoolbox.mdp.RelativeValueIteration(P, R)
mdp_rel_value_iter.run()

mdp_policy_iter_modified = mdptoolbox.mdp.PolicyIterationModified(P, R, 0.99)
mdp_policy_iter_modified.run()


Greedy policy generation

In [112]:
greedy_policy = np.ndarray((S,), dtype=int)
for s in range(S):
    current_max_reward = np.NINF
    current_best_action = None
    for a in range(A):
        if R[s][a] > current_max_reward:
            current_max_reward = R[s][a]
            current_best_action = a
    greedy_policy[s] = current_best_action




Evaluation:
Evaluation method for policies using random starting state, stochastic state transitions and calculating average reward

In [113]:
def eval_policy(policy, iterations):
    # random starting point
    # current_state = np.random.randint(0, S)
    # Basic starting point: Store white, storage is empty
    current_state = 0b001_11_11_11_11
    acc_reward = 0
    for i in range(iterations):
        action = policy[current_state]
        acc_reward += R[current_state][action]
        states = np.arange(0, S, dtype=int)
        current_state = np.random.choice(states, 1, p=P[action][current_state])[0]

    return acc_reward / iterations


In [114]:
print("Greedy AVG Reward: " + str(eval_policy(greedy_policy, 100_000)))
print("Q-Learned MDP AVG Reward: " + str(eval_policy(mdp.policy, 100_000)))
print("Policy Iteration AVG Reward: " + str(eval_policy(mdp_policy_iter.policy, 100_000)))
print("Value Iteration AVG Reward: " + str(eval_policy(mdp_value_iter.policy, 100_000)))
print("Value Iteration GS AVG Reward: " + str(eval_policy(mpd_value_iter_gs.policy, 100_000)))
print("Relative Value Iteration AVG Reward: " + str(eval_policy(mdp_rel_value_iter.policy, 100_000)))
print("Policy Iteration Mofified AVG Reward: " + str(eval_policy(mdp_policy_iter_modified.policy, 100_000)))

Greedy AVG Reward: -10.0
Q-Learned MDP AVG Reward: -39.45068
Policy Iteration AVG Reward: -7.73803
Value Iteration AVG Reward: -7.7554
Value Iteration GS AVG Reward: -7.732
Relative Value Iteration AVG Reward: -7.73857
Policy Iteration Mofified AVG Reward: -7.74901
